In [1]:
from sklearn.feature_extraction.text import HashingVectorizer
from sklearn.feature_extraction.text import CountVectorizer
from sklearn.feature_extraction.text import TfidfTransformer
from sklearn.linear_model import SGDClassifier
from sklearn.model_selection import GridSearchCV
from sklearn.neural_network import MLPClassifier
from sklearn.pipeline import Pipeline
import numpy as np
from time import time

Load the training data

In [3]:

#filename = r'C:\P\Machine Learning\cotoctes\ucici.csv'
filename = r'C:\P\Machine Learning\cotoctes\ucici_full.csv'
import pandas as pd
#training = pd.read_csv(filename,delimiter='|',header=None,names = ['cat', 'data'])
training = pd.read_csv(filename,delimiter='|',header=None,names = ['cat','other', 'data'])
training['yr'] = training['other'].str.slice(1, 3)
training = training[training['yr'].isin(['15','16','17'])]
X_train,X_test = training[0:-5000],training[-5000:]

print(X_train.shape,X_test.shape)
for a, b in zip(X_train.data[0:2], X_train.cat[0:2]):
    print("{0} => {1}".format(a,b))
    
#def size_mb(docs):
#    return sum(len(s.encode('utf-8')) for s in docs) / 1e6

#data_train_size_mb = size_mb(X_train.data)
#data_test_size_mb = size_mb(X_test.data)

#print("%d items - %0.3fMB (training set)" % (
#    len(X_train.data), data_train_size_mb))
#print("%d items - %0.3fMB (test set)" % (
#    len(X_test.data), data_test_size_mb))
print(X_train.head(5))

(58115, 4) (5000, 4)
Náklaďáku praskla pneumatika, převrátil se a zasypal silnici pískem => zpr
Nástupkyně Natalie. Novinářka Suková zkusí navázat na tenisovou dynastii => spo
   cat                            other  \
0  zpr  A170809_114631_praha-zpravy_rsr   
1  spo         A170813_203426_tenis_rou   
2  spo  A170807_070846_sport-basket_ten   
4  zpr        A151226_141456_domaci_pku   
5  zpr   A170303_143744_plzen-zpravy_pp   

                                                data  yr  
0  Náklaďáku praskla pneumatika, převrátil se a z...  17  
1  Nástupkyně Natalie. Novinářka Suková zkusí nav...  17  
2  Rozdílné cesty na ME: juniorky s rekordem, jun...  17  
4  Svět je složitější, než jak nám líčí z Lán, re...  15  
5  Prodala šéfka kočičího spolku vilu pod cenou? ...  17  


In [4]:
parameters = {
    'solver': [('adam')],
    'alpha': (0.001, 0.00001, 0.000001),
    'hidden_layer_sizes': [(4)],
}

In [ ]:
from sklearn.model_selection import ParameterGrid
for p in list(ParameterGrid(parameters)):
    t0 = time()
    print(p)
    pipeline = Pipeline([
        ('vect', HashingVectorizer()),
        ('clf', MLPClassifier(**p)),
        ])
    pipeline.fit(X_train.data, X_train.cat)
    print("Testing {0:.2f} / Training {1:.2f}".format(np.mean(pipeline.predict(X_train.data) == X_train.cat),np.mean(pipeline.predict(X_test.data) == X_test.cat)))
    print("done in %0.3fs" % (time() - t0))

{'alpha': 0.001, 'hidden_layer_sizes': 4, 'solver': 'adam'}
Testing 1.00 / Training 0.95
done in 14805.662s
{'alpha': 1e-05, 'hidden_layer_sizes': 4, 'solver': 'adam'}


C:\Users\v.dekanovsky\AppData\Local\Continuum\Anaconda3\lib\site-packages\sklearn\neural_network\multilayer_perceptron.py:566: UserWarning: Training interrupted by user.
  warnings.warn("Training interrupted by user.")


Testing 0.59 / Training 0.59
done in 174.213s
{'alpha': 1e-06, 'hidden_layer_sizes': 4, 'solver': 'adam'}


In [10]:
filename = r'C:\P\Machine Learning\cotoctes\testovaci.csv'
testing = pd.read_csv(filename,delimiter='|',header=None,names = ['data'])
#with open(r'C:\P\Machine Learning\cotoctes\outputBestMPL.csv','w') as f:
#    wr = csv.writer(f, delimiter='\n')
#    wr.writerow(gs_clf.predict(testing.data))

In [22]:
from sklearn.pipeline import FeatureUnion
from sklearn.linear_model import Perceptron
#from sknn.mlp import MultiLayerPerceptron
pipeline2 = Pipeline([
        ('vect',Pipeline([
                   ('union', FeatureUnion(
                           transformer_list=[
                       ('words', CountVectorizer(analyzer='word',ngram_range=(1,2))),
                       ('chars', CountVectorizer(analyzer='char',ngram_range=(1,3))),
                        ],)),
                   ('tfidf', TfidfTransformer(use_idf=False)),   
                    ])),
        ('clf', Perceptron(max_iter=25)),
        #('clf', MultiLayerPerceptron(max_iter=50,dropout_rate=0.5)),
        ])
pipeline2.fit(X_train.data, X_train.cat)
print("Testing {0:.2f} / Training {1:.2f}".format(np.mean(pipeline2.predict(X_train.data) == X_train.cat),np.mean(pipeline2.predict(X_test.data) == X_test.cat)))

Testing 1.00 / Training 0.93


In [23]:
import csv
with open(r'C:\P\Machine Learning\cotoctes\outputCountPercFull.csv','w') as f:
    wr = csv.writer(f, delimiter='\n')
    wr.writerow(pipeline2.predict(testing.data))

In [24]:
pipeline2.predict(['Pioneer má první interní mechaniku pro přehrávání Ultra HD disků na PC'])

array(['tec'], 
      dtype='<U3')

In [25]:
testing['cat'] = pipeline2.predict(testing.data)
print(testing.head(10))
print(testing.tail(10))

                                                data  cat
0  Celtic znemožnil v boji o Ligu mistrů Astanu, ...  spo
1  Katar obnovuje plné diplomatické styky s Íráne...  zpr
2  Před 100 lety se narodil česko-židovský badate...  tec
3  Pioneer má první interní mechaniku pro přehráv...  tec
4  Studie z Černobylu tvrdí, že radiace zvířatům ...  spo
5   Černý pátek je tu. Jak a kde si počíhat na slevy  tec
6  Karolína Plíšková začne hájit trůn. O číslo 1 ...  spo
7  Od Facebooku nemůžete čekat žádné soukromí, od...  tec
8  Oparům se lze bránit. Pomohou antivirotika i p...  ona
9  Vyhovuje nám, že nás předem odepisují, soudí z...  spo
                                                  data  cat
990  Jsme stříbrní, utěšují se američtí fotbalisté ...  spo
991  Závislost na plastikách mi zničila sexuální ži...  ona
992  Internet, jak jej známe, může skončit, varoval...  tec
993  V případě řízení s firmou FAU mohla být poruše...  zpr
994  Pokrok v IT je obrovský, ale skrytý, řekl šéf ...  spo
99